In [1]:
# ---------------------------------------------------------------------
# Updated Colab cell: DenseNet201 two-stage fine-tuning + robust evaluation
# - Enforces AdamW optimizer by default
# - Removes EarlyStopping so training completes all epochs
# - Adds training curves (loss/accuracy), confusion matrix, ROC, CSV of predictions
# - Saves metrics summary JSON and final model
# ---------------------------------------------------------------------

# ---------------------------
# Imports (top)
# ---------------------------
import os
import zipfile
import math
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from collections import Counter

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize

# ---------------------------
# Configuration — edit as needed
# ---------------------------
DRIVE_MOUNT_POINT = "/content/drive"
ZIP_PATH = "/content/drive/MyDrive/d.zip"   # optional zip on Drive
DATA_ROOT = "/content/dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "d", "train")
VAL_DIR   = os.path.join(DATA_ROOT, "d", "val")

OUT_DIR = "/content/drive/MyDrive/densenet201_results"
CHECKPOINT_HEAD = os.path.join(OUT_DIR, "best_densenet201_head.keras")
CHECKPOINT_FINETUNE = os.path.join(OUT_DIR, "best_densenet201_finetuned.keras")
FINAL_MODEL_PATH = os.path.join(OUT_DIR, "final_densenet201_model.keras")
CSV_PRED = os.path.join(OUT_DIR, "val_predictions_densenet201.csv")
GRAPH_DIR = os.path.join(OUT_DIR, "plots")
METRICS_JSON = os.path.join(OUT_DIR, "training_summary_densenet201.json")

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

# Training schedule (phase1 head, phase2 finetune)
PHASE1_EPOCHS = 21
PHASE2_EPOCHS = 21
TOTAL_EPOCHS = PHASE1_EPOCHS + PHASE2_EPOCHS

# Fine-tuning: number of last base layers to unfreeze in phase2
UNFREEZE_LAST_N = 50   # adjust (20-100 commonly used)

# Optimizer / LR settings
OPTIMIZER_NAME = "adamw"   # enforced to AdamW by default
LR_PHASE1 = 2e-3            # head training LR
LR_PHASE2 = 1e-4            # fine-tuning LR
WEIGHT_DECAY = 1e-6

# Augmentation & MixUp
USE_MIXUP = False          # keep False for class_weight support and simplicity
MIXUP_ALPHA = 0.2

# Mixed precision (optional)
USE_MIXED_PRECISION = False

# ---------------------------
# Mount Drive (Colab)
# ---------------------------
from google.colab import drive
drive.mount(DRIVE_MOUNT_POINT, force_remount=False)

# ---------------------------
# Repro & mixed precision
# ---------------------------
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

if USE_MIXED_PRECISION:
    try:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("Mixed precision enabled.")
    except Exception as e:
        print("Mixed precision not enabled:", e)

# ---------------------------
# Extract dataset if zipped (optional)
# ---------------------------
os.makedirs(DATA_ROOT, exist_ok=True)
if os.path.exists(ZIP_PATH) and not (os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR)):
    print("Extracting dataset zip from Drive to", DATA_ROOT)
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(DATA_ROOT)
    print("Extraction complete.")
else:
    print("Dataset already present or zip not found. DATA_ROOT =", DATA_ROOT)

# ---------------------------
# Compute class counts & class weights
# ---------------------------
def class_counts_from_dir(dirpath):
    counts = {}
    if not os.path.exists(dirpath):
        return counts
    for cls in sorted(os.listdir(dirpath)):
        cls_path = os.path.join(dirpath, cls)
        if os.path.isdir(cls_path):
            counts[cls] = len([f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))])
    return counts

train_counts = class_counts_from_dir(TRAIN_DIR)
val_counts = class_counts_from_dir(VAL_DIR)
print("Train class counts:", train_counts)
print("Val   class counts:", val_counts)

def make_class_weights(counts):
    if not counts:
        return None
    labels = sorted(counts.keys())
    freqs = np.array([counts[k] for k in labels], dtype=np.float32)
    total = freqs.sum()
    weights = (total / (len(labels) * freqs))
    return dict(zip(labels, weights))

class_weights_by_name = make_class_weights(train_counts)
print("Per-class weights by name:", class_weights_by_name)

# ---------------------------
# Build deterministic datasets (image_dataset_from_directory)
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="int",            # keep integer labels so class_weight works
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Detected classes (index->name):", dict(enumerate(class_names)), "NUM_CLASSES=", NUM_CLASSES)

# Convert class weights by name -> index mapping (works only when not using MixUp)
class_weights = None
if class_weights_by_name and (not USE_MIXUP):
    class_weights = {i: float(class_weights_by_name[name]) for i, name in enumerate(class_names) if name in class_weights_by_name}
    print("Class weights (index):", class_weights)
elif USE_MIXUP and class_weights_by_name:
    print("MixUp enabled — class_weight will be ignored (MixUp produces soft labels).")

# ---------------------------
# Preprocessing & augmentation (DenseNet preprocess)
# ---------------------------
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.12),
    tf.keras.layers.RandomZoom(0.12),
    tf.keras.layers.RandomContrast(0.08),
], name="data_augmentation")

def sample_beta_distribution(size, concentration=MIXUP_ALPHA):
    return np.random.beta(concentration, concentration, size)

def mixup(batch_x, batch_y, alpha=MIXUP_ALPHA):
    lam = sample_beta_distribution(batch_x.shape[0]).astype('float32')
    lam_x = lam.reshape(-1, 1, 1, 1)
    lam_y = lam.reshape(-1, 1)
    index = np.random.permutation(batch_x.shape[0])
    mixed_x = batch_x * lam_x + batch_x[index] * (1 - lam_x)
    # produce soft labels (one-hot weighted)
    y1 = tf.one_hot(batch_y, NUM_CLASSES)
    y2 = tf.one_hot(batch_y[index], NUM_CLASSES)
    mixed_y = y1 * lam_y + y2 * (1 - lam_y)
    return mixed_x, mixed_y

def prepare_train(ds, augment=True, mixup_prob=0.0):
    # ds yields (images, labels:int)
    def _map(x, y):
        x = tf.cast(x, tf.float32)
        x = densenet_preprocess(x)
        if augment:
            x = data_augmentation(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    if USE_MIXUP and mixup_prob > 0.0:
        # apply mixup on each batch (tf.numpy_function wrapper)
        def _mixup_np(x, y):
            x_np = x.numpy()
            y_np = y.numpy()
            x_m, y_m = mixup(x_np, y_np, alpha=mixup_prob)
            return x_m.astype('float32'), y_m.astype('float32')
        def _mixup_tf(x, y):
            x_m, y_m = tf.numpy_function(_mixup_np, [x, y], [tf.float32, tf.float32])
            x_m.set_shape([None, IMG_SIZE[0], IMG_SIZE[1], 3])
            y_m.set_shape([None, NUM_CLASSES])
            return x_m, y_m
        ds = ds.map(_mixup_tf, num_parallel_calls=AUTOTUNE)
    else:
        # keep integer labels for sparse loss
        pass
    return ds

def prepare_val(ds):
    def _map(x,y):
        x = tf.cast(x, tf.float32)
        x = densenet_preprocess(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    return ds

train_ds_prepared = prepare_train(train_ds, augment=True, mixup_prob=MIXUP_ALPHA if USE_MIXUP else 0.0)
val_ds_prepared   = prepare_val(val_ds)

# ---------------------------
# Build model: DenseNet201 + head
# ---------------------------
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet201

def build_model(input_shape=(*IMG_SIZE, 3), n_classes=NUM_CLASSES, dropout_rate=0.5):
    base = DenseNet201(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    x = base.output
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(n_classes, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs=base.input, outputs=outputs)
    return model, base

model, base_model = build_model()
model.summary()

# ---------------------------
# Freeze base entirely for phase1
# ---------------------------
for layer in base_model.layers:
    layer.trainable = False

# ---------------------------
# Optimizer selection — AdamW preferred
# ---------------------------
def get_optimizer(name="adamw", lr=LR_PHASE1, wd=WEIGHT_DECAY):
    name = name.lower().strip()
    if name == "adamax":
        return tf.keras.optimizers.Adamax(learning_rate=lr)
    if name == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    if name == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    if name == "adamw":
        try:
            return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
        except Exception:
            try:
                return tf.keras.optimizers.experimental.AdamW(learning_rate=lr, weight_decay=wd)
            except Exception:
                try:
                    import tensorflow_addons as tfa
                    return tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
                except Exception:
                    print("AdamW not available. Falling back to Adamax.")
                    return tf.keras.optimizers.Adamax(learning_rate=lr)
    raise ValueError("Unsupported optimizer: " + name)

# For phase1 we may wish to use a higher lr; for phase2 we'll create a second optimizer with smaller lr
opt_phase1 = get_optimizer(OPTIMIZER_NAME, lr=LR_PHASE1)

# ---------------------------
# Callbacks (NO EarlyStopping)
# ---------------------------
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(GRAPH_DIR, exist_ok=True)

checkpoint_cb_phase1 = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_HEAD, monitor='val_loss', save_best_only=True, verbose=1
)
checkpoint_cb_phase2 = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_FINETUNE, monitor='val_loss', save_best_only=True, verbose=1
)
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6, verbose=1)
# EarlyStopping intentionally removed so training completes all epochs
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(OUT_DIR, "tb_logs"))

# ---------------------------
# Compile & train Phase 1 (head only)
# ---------------------------
# Loss choice:
if USE_MIXUP:
    # MixUp produces soft labels -> categorical_crossentropy and labels must be one-hot/soft vectors
    loss_fn = 'categorical_crossentropy'
else:
    loss_fn = 'sparse_categorical_crossentropy'  # preserves compatibility with class_weight

model.compile(optimizer=opt_phase1, loss=loss_fn, metrics=['accuracy'])
model.build((None, IMG_SIZE[0], IMG_SIZE[1], 3))

print("\n=== PHASE 1: training head (base frozen) ===")
history_phase1 = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared,
    epochs=PHASE1_EPOCHS,
    callbacks=[checkpoint_cb_phase1, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights if (not USE_MIXUP) else None,
    verbose=1
)

# ---------------------------
# Phase 2: unfreeze last N layers & fine-tune
# ---------------------------
# Unfreeze last UNFREEZE_LAST_N layers
if UNFREEZE_LAST_N > 0:
    # set last N layers trainable
    for layer in base_model.layers[:-UNFREEZE_LAST_N]:
        layer.trainable = False
    for layer in base_model.layers[-UNFREEZE_LAST_N:]:
        layer.trainable = True
else:
    # Unfreeze all except BatchNorm layers (safer)
    for layer in base_model.layers:
        layer.trainable = True
    for layer in base_model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
opt_phase2 = get_optimizer(OPTIMIZER_NAME, lr=LR_PHASE2)
model.compile(optimizer=opt_phase2, loss=loss_fn, metrics=['accuracy'])

print("\n=== PHASE 2: fine-tuning last %d layers ===" % UNFREEZE_LAST_N)
history_phase2 = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared,
    epochs=TOTAL_EPOCHS,
    initial_epoch=PHASE1_EPOCHS,
    callbacks=[checkpoint_cb_phase2, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights if (not USE_MIXUP) else None,
    verbose=1
)

# ---------------------------
# Combine histories & plot training curves
# ---------------------------

def combine_histories(h1, h2):
    out = {}
    for k in h1.history.keys():
        out[k] = h1.history[k] + h2.history.get(k, [])
    for k in h2.history.keys():
        if k not in out:
            out[k] = h2.history[k]
    return out

combined = combine_histories(history_phase1, history_phase2)

def plot_and_save_training_curves(history_dict, out_dir):
    epochs = range(1, len(history_dict.get('loss', [])) + 1)
    # Accuracy
    if 'accuracy' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['accuracy'], label='train_acc')
        if 'val_accuracy' in history_dict:
            plt.plot(epochs, history_dict['val_accuracy'], label='val_acc')
        plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Training & Validation Accuracy')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "accuracy_curve.png")
        plt.savefig(p); plt.close()
        print("Saved accuracy plot to:", p)
    # Loss
    if 'loss' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['loss'], label='train_loss')
        if 'val_loss' in history_dict:
            plt.plot(epochs, history_dict['val_loss'], label='val_loss')
        plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training & Validation Loss')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "loss_curve.png")
        plt.savefig(p); plt.close()
        print("Saved loss plot to:", p)

plot_and_save_training_curves(combined, GRAPH_DIR)

# ---------------------------
# Save final model
# ---------------------------
try:
    model.save(FINAL_MODEL_PATH)
    print("Saved final model to:", FINAL_MODEL_PATH)
except Exception as e:
    print("Final save (.keras) failed:", e)
    try:
        fallback = FINAL_MODEL_PATH.replace(".keras", ".h5")
        model.save(fallback)
        print("Saved fallback model to:", fallback)
    except Exception as e2:
        print("Final save also failed:", e2)

# ---------------------------
# Load best checkpoint for evaluation (prefer finetuned then head)
# ---------------------------
preferred = CHECKPOINT_FINETUNE if os.path.exists(CHECKPOINT_FINETUNE) else CHECKPOINT_HEAD if os.path.exists(CHECKPOINT_HEAD) else FINAL_MODEL_PATH
print("Attempting to load best model from:", preferred)
try:
    eval_model = tf.keras.models.load_model(preferred)
    print("Loaded model for evaluation:", preferred)
except Exception as e:
    print("Could not load checkpoint; using in-memory model. Error:", e)
    eval_model = model

# ---------------------------
# Robust evaluation: predictions, confusion matrix, ROC, CSV
# ---------------------------

# Reconstruct validation file list in deterministic (class/filename) order used by image_dataset_from_directory
val_filepaths = []
for cls in class_names:
    cls_dir = os.path.join(VAL_DIR, cls)
    if not os.path.isdir(cls_dir):
        continue
    files = sorted([f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))])
    for fname in files:
        val_filepaths.append(os.path.join(cls, fname))
print(f"Reconstructed {len(val_filepaths)} validation file paths.")

# Build prediction dataset (shuffle=False) matching order
val_pred_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)
val_pred_ds_pp = val_pred_ds.map(lambda x,y: (densenet_preprocess(tf.cast(x, tf.float32)), y)).prefetch(AUTOTUNE)

# Predict
pred_probs = eval_model.predict(val_pred_ds_pp, verbose=1)

# Extract y_true robustly (concatenate labels produced by dataset)
y_true = np.concatenate([y.numpy() for x,y in val_pred_ds], axis=0)
y_pred = np.argmax(pred_probs, axis=1)

# Sanity checks
if pred_probs.shape[0] != len(y_true):
    print("WARNING: preds rows != true labels:", pred_probs.shape[0], len(y_true), len(val_filepaths))

# Map index->label
index_to_label = {i: name for i, name in enumerate(class_names)}
target_names = [index_to_label[i] for i in sorted(index_to_label.keys())]

# Accuracy & confusion matrix
acc = accuracy_score(y_true, y_pred) if len(y_true) > 0 else 0.0
print(f"\nValidation accuracy: {acc*100:.2f}%")

cm = confusion_matrix(y_true, y_pred, labels=sorted(index_to_label.keys()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.title("Confusion Matrix")
cm_path = os.path.join(GRAPH_DIR, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()
print("Saved confusion matrix to:", cm_path)

print("\nClassification report:\n")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

# ROC per-class (one-vs-rest)
roc_path = os.path.join(GRAPH_DIR, "roc_curve.png")
try:
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    fpr, tpr, roc_auc = {}, {}, {}
    present_classes = []
    for i in range(NUM_CLASSES):
        if np.sum(y_true_bin[:, i]) == 0:
            print(f"Skipping ROC for class {i} ({index_to_label.get(i,'?')}) — no positives in y_true")
            continue
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        present_classes.append(i)

    plt.figure(figsize=(9,8))
    for i in present_classes:
        plt.plot(fpr[i], tpr[i], label=f"{index_to_label.get(i, i)} (AUC={roc_auc[i]:.3f})")
    plt.plot([0,1],[0,1],'k--', label='Chance')
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate"); plt.title("ROC Curves (one-vs-rest)")
    plt.legend(loc='lower right'); plt.grid(True)
    plt.savefig(roc_path)
    plt.close()
    print("Saved ROC plot to:", roc_path)
except Exception as e:
    print("ROC plotting skipped due to:", e)

# For binary classification, also compute single AUC using positive class probs (index 1)
auc_score = None
try:
    if NUM_CLASSES == 2 and pred_probs.shape[1] >= 2 and len(y_true) > 0:
        auc_score = roc_auc_score(y_true, pred_probs[:, 1])
        print(f"Binary ROC AUC (class '{index_to_label.get(1,1)}' as positive): {auc_score:.4f}")
except Exception as e:
    print("Binary AUC computation failed:", e)

# Save per-sample CSV (filename, true_label, pred_label, per-class probs)
rows = []
min_len = min(len(val_filepaths), len(y_true), pred_probs.shape[0])
if min_len < len(y_true):
    print("NOTE: filename count differs from true/pred count — using min_len =", min_len)

for i in range(min_len):
    filename = val_filepaths[i] if i < len(val_filepaths) else f"unknown_{i}"
    rows.append({
        "filename": filename,
        "true_label": index_to_label[int(y_true[i])],
        "pred_label": index_to_label[int(y_pred[i])],
        **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
    })

# append leftover true entries safely
if len(y_true) > min_len:
    for i in range(min_len, len(y_true)):
        rows.append({
            "filename": f"unknown_{i}",
            "true_label": index_to_label[int(y_true[i])],
            "pred_label": index_to_label[int(y_pred[i])],
            **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
        })


df = pd.DataFrame(rows)
df.to_csv(CSV_PRED, index=False)
print("Saved per-sample predictions to:", CSV_PRED)

# ---------------------------
# Save summary metrics & training info
# ---------------------------
metrics_summary = {
    "num_classes": int(NUM_CLASSES),
    "class_names": class_names,
    "train_counts": train_counts,
    "val_counts": val_counts,
    "final_validation_accuracy": float(acc) if 'acc' in locals() else None,
    "binary_auc_positive_class_1": float(auc_score) if auc_score is not None else None,
    "phase1_epochs": int(PHASE1_EPOCHS),
    "phase2_epochs": int(PHASE2_EPOCHS),
    "total_epochs": int(TOTAL_EPOCHS),
    "optimizer": OPTIMIZER_NAME,
    "learning_rate_phase1": LR_PHASE1,
    "learning_rate_phase2": LR_PHASE2,
    "class_weights": class_weights,
    "artifacts": {
        "checkpoint_head": CHECKPOINT_HEAD,
        "checkpoint_finetune": CHECKPOINT_FINETUNE,
        "final_model": FINAL_MODEL_PATH,
        "predictions_csv": CSV_PRED,
        "confusion_matrix": cm_path,
        "roc_curve": roc_path,
        "accuracy_curve": os.path.join(GRAPH_DIR, "accuracy_curve.png"),
        "loss_curve": os.path.join(GRAPH_DIR, "loss_curve.png")
    }
}

with open(METRICS_JSON, "w") as f:
    json.dump(metrics_summary, f, indent=2)
print("Saved training summary JSON to:", METRICS_JSON)

# ---------------------------
# Final save (copy)
# ---------------------------
try:
    eval_model.save(FINAL_MODEL_PATH)
    print("Final model saved to:", FINAL_MODEL_PATH)
except Exception as e:
    print("Final model save failed:", e)

print("\nAll evaluation artifacts saved under:", OUT_DIR)


Mounted at /content/drive
Extracting dataset zip from Drive to /content/dataset
Extraction complete.
Train class counts: {'benign': 1440, 'malignant': 1197}
Val   class counts: {'benign': 360, 'malignant': 300}
Per-class weights by name: {'benign': np.float32(0.915625), 'malignant': np.float32(1.1015037)}
Found 2637 files belonging to 2 classes.
Found 660 files belonging to 2 classes.
Detected classes (index->name): {0: 'benign', 1: 'malignant'} NUM_CLASSES= 2
Class weights (index): {0: 0.9156249761581421, 1: 1.1015037298202515}
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 18,822,978 (71.80 MB)

 Trainable params: 18,589,570 (70.91 MB)

 Non-trainable params: 233,408 (911.75 KB)


=== PHASE 1: training head (base frozen) ===
Epoch 1/21
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.7654 - loss: 0.6171
Epoch 1: val_loss improved from inf to 0.49395, saving model to /content/drive/MyDrive/densenet201_results/best_densenet201_head.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.7658 - loss: 0.6158 - val_accuracy: 0.7788 - val_loss: 0.4940 - learning_rate: 0.0020
Epoch 2/21
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.8726 - loss: 0.2922
Epoch 2: val_loss improved from 0.49395 to 0.39824, saving model to /content/drive/MyDrive/densenet201_results/best_densenet201_head.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 14s 167ms/step - accuracy: 0.8725 - loss: 0.2924 - val_accuracy: 0.8136 - val_loss: 0.3982 - learning_rate: 0.0020
Epoch 3/21
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9018 - loss: 0.2322
Epoch 3: val_loss improved from 0.39824 to 0.39036, saving model to /content/drive/MyDrive/densenet201_results/best_densenet201_head.keras
8